## Tools 案例

In [7]:
from pydantic import BaseModel, Field
from typing import List
from typing_extensions import Literal

tools = [
    {
        "type": "function",
        "function": {
            "name": "Ticket",
            "description": "根据用户提供的信息查询火车时刻",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {
                        "description": "要查询的火车日期",
                        "title": "Date",
                        "type": "string",
                    },
                    "departure": {
                        "description": "出发城市或车站",
                        "title": "Departure",
                        "type": "string",
                    },
                    "destination": {
                        "description": "要查询的火车日期",
                        "title": "Destination",
                        "type": "string",
                    },
                },
                "required": ["date", "departure", "destination"],
            },
        },
    }
]

messages = [
    {
        "role": "user",
        "content": "你能帮我查一下2024年1月1日从北京南站到上海的火车票吗？"
    }
]

from zhipuai import ZhipuAI
client = ZhipuAI(api_key="填入你的key")

response = client.chat.completions.create(
    model="glm-4-plus", # 请填写您要调用的模型名称
    messages=messages,
    tools=tools,
    tool_choice="auto",
)
print(response.choices[0].message)

CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_9051234730097475169', function=Function(arguments='{"date": "2024-01-01", "departure": "北京南站", "destination": "上海"}', name='Ticket'), type='function', index=0)])


In [8]:
tools = [
    {
        "type": "function",
        "function": {
            "name": Ticket.schema()['title'],
            "description": Ticket.schema()['description'],
            "parameters": {
                "type": "object",
                "properties": Ticket.schema()['properties'],
                "required": Ticket.schema()['required'],
            },
        }
    }
]

## Tools 自动解析

In [10]:
class GLMAgent:
    def __init__(self, model_name: str):
        self.model_name = model_name

    def call(self, user_prompt, response_model):
        messages = [
            {
                "role": "user",
                "content": user_prompt
            }
        ]
        tools = [
            {
                "type": "function",
                "function": {
                    "name": response_model.schema()['title'],
                    "description": response_model.schema()['description'],
                    "parameters": {
                        "type": "object",
                        "properties": response_model.schema()['properties'],
                        "required": response_model.schema()['required'],
                    },
                }
            }
        ]

        response = client.chat.completions.create(
            model=self.model_name,
            messages=messages,
            tools=tools,
            tool_choice="auto",
        )
        try:
            arguments = response.choices[0].message.tool_calls[0].function.arguments
            return response_model.model_validate_json(arguments)
        except:
            print('ERROR', response.choices[0].message)
            return None


In [11]:
class Ticket(BaseModel):
    """根据用户提供的信息查询火车时刻"""
    date: str = Field(description="要查询的火车日期")
    departure: str = Field(description="出发城市或车站")
    destination: str = Field(description="要查询的火车日期")

GLMAgent(model_name = 'glm-4-plus').call("你能帮我查一下2024年1月1日从北京南站到上海的火车票吗？", Ticket)

Ticket(date='2024-01-01', departure='北京南站', destination='上海')

## 中文分词

In [80]:
class Text(BaseModel):
    """抽取句子中的的单词，进行文本分词"""
    keyword: List[str] = Field(description="单词")

GLMAgent(model_name = 'glm-4-plus').call('阿水是强哥的好朋友。谢大脚是长贵的老公。', Text)

Text(keyword=['阿水', '强哥', '好朋友', '谢大脚', '长贵', '老公'])

## 文本分类

In [62]:
class Text(BaseModel):
    """分析文本的情感"""
    sentiment: Literal["正向", "反向"] = Field(description="情感类型")

GLMAgent(model_name = 'glm-4-air').call('我今天很开心。', Text)

Text(sentiment='正向')

In [63]:
class Text(BaseModel):
    """分析文本的情感"""
    sentiment: Literal["postivate", "negative"] = Field(description="情感类型")

GLMAgent(model_name = 'glm-4-air').call('我今天很开心。', Text)

Text(sentiment='postivate')

## 文本匹配

In [66]:
class Text(BaseModel):
    """判断句子是否语义相近"""
    score:float = Field(description="文本相似度，介于0与1。0代表不相似，1代表相似")

GLMAgent(model_name = 'glm-4-air').call('我今天很开心 与 我今天不开心。', Text)

Text(score=0.0)

In [67]:
GLMAgent(model_name = 'glm-4-air').call('我今天很开心 与 我今天超级开心。', Text)

Text(score=0.8)

## 实体抽取

In [30]:
class Text(BaseModel):
    """抽取实体"""
    person: List[str] = Field(description="人名")
    location: List[str] = Field(description="地名")

GLMAgent(model_name = 'glm-4-air').call('今天我和徐也也去海淀吃饭，强哥也去了。', Text)

Text(person=['我', '徐也也', '强哥'], location=['海淀'])

## 关系抽取

In [12]:
class Text(BaseModel):
    """抽取句子中所有实体之间的关系"""
    source_person: List[str] = Field(description="原始实体")
    target_person: List[str] = Field(description="目标实体")
    relationship: List[Literal["朋友", "亲人", "同事"]] = Field(description="待选关系")

GLMAgent(model_name = 'glm-4-air').call('阿水是强哥的好朋友。谢大脚是长贵的老公。', Text)

Text(source_person=['强哥'], target_person=['阿水'], relationship=['朋友'])

## 文本摘要

In [18]:
class Text(BaseModel):
    """抽取句子的摘要"""
    abstract: str = Field(description="摘要结果")

GLMAgent(model_name = 'glm-4-plus').call("20年来，中国探月工程从无到有、从小到大、从弱到强。党的十八大后，一个个探月工程任务连续成功，不断刷新世界月球探测史的中国纪录嫦娥三号实现我国探测器首次地外天体软着陆和巡视探测，总书记肯定“在人类攀登科技高峰征程中刷新了中国高度”；", Text)

Text(abstract='中国探月工程20年发展，连续成功任务刷新世界纪录，总书记肯定成就。')

## 关键词提取

In [78]:
class Text(BaseModel):
    """抽取句子中的关键词"""
    keyword: str = Field(description="关键词")

GLMAgent(model_name = 'glm-4-plus').call('阿水是强哥的好朋友。谢大脚是长贵的老公。', Text)

Text(keyword='阿水')

## 复杂案例

In [61]:
class Text(BaseModel):
    """文本问答内容解析"""
    search: bool = Field(description="是否需要搜索")
    keywords: List[str] = Field(description="待选关键词")
    intent: Literal["查询客服问题", "查询产品问题", "查询系统问题", "其他"] = Field(description="意图")

GLMAgent(model_name = 'glm-4-plus').call('汽车发动和轮胎出故障了，如何处理？', Text)

Text(search=True, keywords=['汽车发动故障', '轮胎故障', '处理方法'], intent='查询产品问题')

In [82]:
class Text(BaseModel):
    """文本问答内容解析"""
    time: List[str] = Field(description="时间")
    particate: List[str] = Field(description="选手")
    competition: List[str] = Field(description="赛事名称")

GLMAgent(model_name = 'glm-4-plus').call('2月8日上午北京冬奥会自由式滑雪女子大跳台决赛中中国选手谷爱凌以188.25分获得金牌！2022语言与智能技术竞赛由中国中文信息学会和中国计算机学会联合主办。', Text)

Text(time=['2月8日上午'], particate=['谷爱凌'], competition=['北京冬奥会自由式滑雪女子大跳台决赛', '2022语言与智能技术竞赛'])